# Everymind - FireData
Neste notebook, apresentamos a descrição do processo de desenvolver um modelo preditivo de classificação capaz de identificar quais funcionários têm tendências a sair ou permanecer na empresa.

# Integrantes

*   Alexandre Fonseca de Souza
*   Bruno Otavio Bezerra de Meira
*   Filipi Enzo Siqueira Kikuchi
*   Gabriela de Morais da Silva
*   João Pedro Sartori Garcia de Alcaraz
*   Lucas Conti Pereira
*   Sofia Moreiras Pimazzoni


Primeiramente, utilizaremos o arquivo que contém a base de dados dos colaboradores para a future engineering.

#1.Preparação do algoritmo
Para que o algoritmo funcione da forma desejada, primeiro instalamos e importamos funções e bibliotecas que auxiliam na execução de algumas tarefas. Todas as importações estão listadas abaixo:

In [70]:
# Instalando o moment
!pip install moment

# Importação de bibliotecas
# Pandas
import pandas as pd

# scikitlearn
from sklearn.preprocessing import LabelEncoder

# Datetime
from datetime import datetime
from datetime import date

# Moment
import moment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1.1 Importação dos dados
Para que o algoritmo consiga performar, precisamos importar o arquivo contendo os dados, a fim de tratá-los e posteriormente fazer analises mais acuradas. Como possuímos três tabelas com dados diferentes, separamo-nos em 3 dataframes diferentes.

In [71]:
# Importando o arquivo que contém a base de dados

# Dataframe da tabela "Everymind"
df_everymind = pd.read_excel('/content/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', sheet_name = "Everymind")

# Dataframe da tabela "Reconhecimento"
df_reconhecimento = pd.read_excel('/content/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', sheet_name = "Reconhecimento")

# Dataframe da tabela "Reconhecimento"
df_ambiente = pd.read_excel('/content/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', sheet_name = "Ambiente de Trabalho 27.07")

# 2.Remover ou substituir registros em branco

In [72]:
#------------------------------------Tabela Everymind--------------------------

# Verifica se dentro da coluna "Dt Saisa" tem cédulas vazias e caso tenha, adiciona a data de hoje.
df_saida = df_everymind['Dt Saida'].isnull()

for i in range(0,len(df_saida)):
  if df_saida[i] == True:
    df_everymind['Dt Saida'][i] = pd.Timestamp.today()

# Verifica se dentro da coluna "Tipo Saida" tem cédulas vazias e caso tenha, adciona "ColaboradorAtivo".

df_tipo_bool = df_everymind['Tipo Saida'].isnull()
df_tipo_aux = df_everymind['Tipo Saida']

for i in range(0,len(df_tipo_bool)):
  if df_tipo_bool[i] == True:
    df_tipo_bool[i] = "ColaboradorAtivo"
  if df_tipo_bool[i] == False: 
    df_tipo_bool[i] = (df_tipo_aux[i])


#------------------------------------Tabela Ambiente de trabalho--------------------------

# Verifica se dentro das colunas de porcentagem de satisfação tem cédulas vazias e caso tenha, adiciona o 0.
keywords = ['Pulou', 'Muito Insatisfeito', 'Insatisfeito', 'Neutro', 'Satisfeito', 'Muito Satisfeito']

for i in keywords:
  
  df_ambiente_satisfaction = df_ambiente[i].isnull()

  for j in range(0,len(df_ambiente_satisfaction)):
    if df_ambiente_satisfaction[j] == True:
      df_ambiente[i][j] = "0"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#3. Formatação
 Nesta seção, vamos tratar os dados a fim de padronizá-los para que sejam aceitos e melhor utilizados pelo algoritmo a partir de funções que modificam a forma do dado. No momento, estamos trabalhando com alguns tipos de dados, sendo eles:



*   Dados relacionadas a tempo e numeros
*   Dados relaciondados a nome



3.1 Variáveis relacionadas a tempo e numeros
As manipulações para dados do tipo data envolvem a conversão do formato para o padrão "YYYY-MM-DD" (Ex: 2020-09-14) e a inclusão de valores em cédulas vazias para efeitos de cálculo (Na coluna "Dt Saida", quando não consta a data de saída, atribuimos o dia atual para podermos calcular o número de dias na empresa). Sofrem essa alteração os dados presentes nas colunas "Dt Admissao", "Dt Nascimento" e "Dt Saida". As transformações seguem abaixo:

In [73]:
#------------------------------------Tabela Everymind--------------------------

# Formata a data de admissao no formato ano/mes/dia
for i in range(0, len(df_everymind['Dt Admissao'])):

  df_everymind['Dt Admissao'][i] = str(df_everymind['Dt Admissao'][i]).replace('00:00:00', '')
  df_everymind['Dt Admissao'][i] = str(df_everymind['Dt Admissao'][i]).replace('/', '-')
  df_everymind['Dt Admissao'][i] = moment.date(df_everymind['Dt Admissao'][i]).format("YYYY-MM-DD")

df_everymind['Dt Admissao'] = pd.to_datetime(df_everymind['Dt Admissao'], format="%Y/%m/%d")

# Formata a data de nascimento no formato ano/mes/dia

for i in range(0, len(df_everymind['Dt Nascimento'])):

  df_everymind['Dt Nascimento'][i] = moment.date(df_everymind['Dt Nascimento'][i]).format("YYYY-MM-DD")

df_everymind['Dt Nascimento'] = pd.to_datetime(df_everymind['Dt Nascimento'], format="%Y/%m/%d")


# Formata a data de saída, sendo que se o valor for vazio, atribui a data atual como data de saída

df_everymind['Dt Saida'] = pd.to_datetime(df_everymind['Dt Saida'], format="%Y/%m/%d")

#------------------------------------Tabela Reconhecimento--------------------------

# Coloca a coluna "Data de Admissao" no formato ano/mes/dia

for i in range(0, len(df_reconhecimento['Data de Admissão'])):

  df_reconhecimento['Data de Admissão'][i] = str(df_reconhecimento['Data de Admissão'][i]).replace('00:00:00', '')
  df_reconhecimento['Data de Admissão'][i] = str(df_reconhecimento['Data de Admissão'][i]).replace('/', '-')
  df_reconhecimento['Data de Admissão'][i] = moment.date(df_reconhecimento['Data de Admissão'][i]).format("YYYY-MM-DD")

df_reconhecimento['Data de Admissão'] = pd.to_datetime(df_reconhecimento['Data de Admissão'], format="%Y/%m/%d")

# Formata a data de Vigência no formato ano/mes/dia
for i in range(0, len(df_reconhecimento['Data Vigência'])):
  
  df_reconhecimento['Data Vigência'][i] = str(df_reconhecimento['Data Vigência'][i]).replace('00:00:00', '')
  df_reconhecimento['Data Vigência'][i] = str(df_reconhecimento['Data Vigência'][i]).replace('/', '-')
  df_reconhecimento['Data Vigência'][i] = moment.date(df_reconhecimento['Data Vigência'][i]).format("YYYY-MM-DD")

df_reconhecimento['Data Vigência'] = pd.to_datetime(df_reconhecimento['Data Vigência'], format="%Y/%m/%d")


#------------------------------------Tabela Ambiente de trabalho--------------------------


for i in keywords:
  for j in range(0,len(df_ambiente[i])):
    df_ambiente[i][j] = df_ambiente[i][j].replace('%', '')
  df_ambiente[i] = df_ambiente[i].astype(float)
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A valu

##3.2 Variáveis relacionadas a nomes

Para que o programa funcione corretamente, todos os dados que contêm nomes e/ou informações baseadas em texto foram padronizados removendo todos os espaços em branco (Exemplo: "Funcional Pl" vira "FuncionalPl").



In [74]:
# ------------------------------------Tabela Everymind--------------------------


# Pega todas as colunas do dataframe com type "object" e tira todos os espaços em branco
dtg = df_everymind.select_dtypes(include = "object").columns

for j in dtg:
  df_everymind[j] = df_everymind[j].str.replace(' ', '')

# Cria uma coluna nova
df_everymind['PessoaColaboradora'] = ""

#Pega a coluna "Nome completo" tira o texto, deixa apenas o número do colaborador e adiciona na coluna criada "PessoaColaboradora" 

for i in range(0, len(df_everymind['Nome Completo'])):
  df_everymind['PessoaColaboradora'][i] = str(df_everymind['Nome Completo'][i]).replace('PessoaColaboradora', '')


# ------------------------------------Tabela Reconhecimento--------------------------


# Pega todas as colunas com type "object" e tira todos os espaços para ficar padronizado
df_reconhecimento_types = df_reconhecimento.select_dtypes(include = "object").columns

for j in df_reconhecimento_types:
  df_reconhecimento[j] = df_reconhecimento[j].str.replace(' ', '')

# Cria uma nova coluna "Pessoa colaboradora"
df_reconhecimento['PessoaColaboradora'] = ""

# Transforma a coluna "Codinome" em number, tirando os textos e colocando o numero do colaborador na coluna "PessoaColaboradora"
for i in range(0, len(df_reconhecimento['Codinome'])):
  df_reconhecimento['PessoaColaboradora'][i] = str(df_reconhecimento['Codinome'][i]).replace('PessoaColaboradora', '')


# ------------------------------------Tabela Ambiente de Trabalho 27.07--------------------------


# Pega todas as colunas com type "object" e tira todos os espaços para ficar padronizado
df_ambiente_types = df_ambiente.select_dtypes(include = "object").columns

for j in df_ambiente_types:
  df_ambiente[j] = df_ambiente[j].str.replace(' ', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#4. Derivação de novos atributos

A partir da limpeza dos dados e do relacionamento entre eles, conseguimos gerar novos elementos que facilitam as analises e auxiliam na tomada de decisões da IA. A partir dos dados obtidos, obtivemos os seguintes atributos:


*   Número de dias na empresa
*   Idade do colaborador
*   Dias para reconhecimento entre data de admissão e vigência




In [75]:
# ------------------------------------Tabela Everymind--------------------------

# Função que calcula os dias entre duas datas
def numOfDays(date1, date2):
    return (date2-date1).days

# Cria uma coluna de idade e uma de numero de dias na empresa na empresa 
df_everymind['Idade'] = ""
df_everymind['NumeroMeses'] = ""

# Faz o calculo da idade e armazena 
for i in range(0, len(df_everymind['Dt Nascimento'])):
  if df_everymind['Tipo Saida'][i] == str('ColaboradorAtivo'):
    if date.today().month>=(df_everymind['Dt Nascimento'][i]).month:
      df_everymind['Idade'][i] = date.today().year - (df_everymind['Dt Nascimento'][i]).year
    else:
      df_everymind['Idade'][i] = (date.today().year - (df_everymind['Dt Nascimento'][i]).year)-1
  else:
    if df_everymind['Dt Saida'][i].month>=(df_everymind['Dt Nascimento'][i]).month:
      df_everymind['Idade'][i] = date.today().year - (df_everymind['Dt Nascimento'][i]).year
    else:
      df_everymind['Idade'][i] = (df_everymind['Dt Saida'][i].year - df_everymind['Dt Nascimento'][i].year)-1

# Faz o calculo do numero de meses na empresa de cada colaborador

for i in range(0, len(df_everymind['Dt Admissao'])):
  if df_everymind['Tipo Saida'][i] == str('Ativo'):
    df_everymind['NumeroMeses'][i] = (((date.today().year - df_everymind['Dt Admissao'][i].year)*12)+date.today().month - df_everymind['Dt Admissao'][i].month)
  else:
    df_everymind['NumeroMeses'][i] = (df_everymind['Dt Saida'][i].year - df_everymind['Dt Admissao'][i].year)*12+df_everymind['Dt Saida'][i].month - df_everymind['Dt Admissao'][i].month
    
# ------------------------------------Tabela Reconhecimento--------------------------


#Cria uma coluna "DiasParaReconhecimento"
df_reconhecimento['DiasParaReconhecimento'] = ""

#Calcula os dias que o empregador entrou na empresa até receber reconhecimento
date_reconhecimento1 = df_reconhecimento['Data de Admissão']
date_reconhecimento2 = df_reconhecimento['Data Vigência']
for i in range(0, len(date_reconhecimento2)):
  df_reconhecimento['DiasParaReconhecimento'][i] = str(numOfDays(date_reconhecimento1[i], date_reconhecimento2[i]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to b

#5. Encoders

As variáveis do tipo _string_ são reconhecidas com dificuldade pelo modelo, por conta disso, é necessário transformá-las em números (tipo _int_ e _float_). Existem duas maneiras de se fazer o "encoding", sendo elas:


* Label Encoding
* One Hot Encoding

##5.1 Label Encoding

O Label Encoding transforma as features em números ordenados, ou seja, em ordem. É normalmente usada quando uma feature é mais importante que outra. No projeto a única que identificamos que possui ordem é a escolaridade (na tabela Everymind).

In [76]:
# Cria um novo Data Frame para a coluna de Escolaridade e raliza o processo de Label Encoding
data_frame_esco = pd.DataFrame(df_everymind, columns=["Escolaridade "]) 
df_esco_atual = data_frame_esco.replace(['EnsinoMédioIncompleto',
                                         'EnsinoMédio', 
                                         'Técnico', 
                                         'Superiorincompleto', 
                                         'Graduação', 
                                         'PósGraduação', 
                                         'Mestrado'], 
                                         [0,1,2,3,4,5,6])

##5.2 One Hot Encoding

O One Hot Encoding cria uma coluna para cada valor e faz uma atribuição do valor 1 (um) para a coluna correspondente da amostra e 0 (zero) para as outras. Dessa forma, uma linha tem apenas um valor. Essa forma de _encoding_ costuma ser mais utilizada, pois não atribui uma ordem às features. Nós usamos o One Hot Encoding em todas as tabelas.

In [77]:
# ------------------------------------Tabela Everymind--------------------------

# A função "get_dummies()" realiza o One Hot Encoding da coluna desejada. 
one_hot_cargos = pd.get_dummies(df_everymind["Cargo"])
one_hot_saida = pd.get_dummies(df_everymind["Tipo Saida"])
one_hot_genero = pd.get_dummies(df_everymind["Genero"])
one_hot_etnia = pd.get_dummies(df_everymind["Etnia"])
one_hot_civil = pd.get_dummies(df_everymind["Estado Civil"]) 
one_hot_estado = pd.get_dummies(df_everymind["Estado"])
one_hot_cidade = pd.get_dummies(df_everymind["Cidade"])
one_hot_area = pd.get_dummies(df_everymind["Area"])
one_hot_colaborador = pd.get_dummies(df_everymind["PessoaColaboradora"])


# ------------------------------------Tabela Reconhecimento--------------------------

# A mesma função acima se aplica nessa tabela
one_hot_codinome = pd.get_dummies(df_reconhecimento["Codinome"])
one_hot_situacao = pd.get_dummies(df_reconhecimento["Situação"])
one_hot_novocargo = pd.get_dummies(df_reconhecimento["Novo Cargo"])
one_hot_motivo = pd.get_dummies(df_reconhecimento["Motivo"])
one_hot_funcao = pd.get_dummies(df_reconhecimento["Alterou Função"])

# ------------------------------------Tabela Ambiente de trabalho--------------------------

one_hot_div = pd.get_dummies(df_ambiente["Divisao"])
one_hot_pilar = pd.get_dummies(df_ambiente["Pilar"])
one_hot_factor = pd.get_dummies(df_ambiente["Fator"])
one_hot_question = pd.get_dummies(df_ambiente["Pergunta"])
one_hot_reliabilityRate = pd.get_dummies(df_ambiente["Taxa de Confiabilidade"])

one_hot_div

,AMS&PS,Analytics&Integration,CPG&RetailI,CPG&RetailII,Commerce,Core&IndustriasI,Core&IndustriasII,Everymind,MktCloud,People&Culture,Vendas
0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1689,0,0,0,0,0,0,0,0,0,0,1
1690,0,0,0,0,0,0,0,0,0,0,1
1691,0,0,0,0,0,0,0,0,0,0,1
1692,0,0,0,0,0,0,0,0,0,0,1


#6. Criação de novas tabelas

Para facilitar o desenvolvimento do projeto, foi decidido criar tabelas novas com as modificações feitas.

6.1. Armazenando os dados em variaveis

In [78]:
# ------------------------------------Tabela Everymind--------------------------

collaborator = df_everymind['PessoaColaboradora']
registration = df_everymind['Matrícula']
age = df_everymind['Idade']
companyTime = df_everymind['NumeroMeses']
monthlySalary = df_everymind['Salario Mês']

# ------------------------------------Tabela Reconhecimento--------------------------

registrationRecognition = df_reconhecimento['Matricula']
registrationRecognition = df_reconhecimento['PessoaColaboradora']
daysRecognition = df_reconhecimento['DiasParaReconhecimento']
newSalary =  df_reconhecimento['Novo Salario']

# ------------------------------------Tabela Ambiente de trabalho--------------------------

punctuation = df_ambiente['Pontuação']
subpunctuation = df_ambiente['Pontuação.1']
searchSkip = df_ambiente['Pulou']
veryUnsatisfied = df_ambiente['Muito Insatisfeito']
unsatisfied = df_ambiente['Insatisfeito']
neutral = df_ambiente['Neutro']
satisfied = df_ambiente['Satisfeito']
verySatisfied = df_ambiente['Muito Satisfeito']

6.2. Concatenando os dados na nova tabela

In [79]:
# ------------------------------------Tabela Everymind--------------------------

# Cria uma tabela nova com os dados formatados
newData = pd.DataFrame()
newData = pd.concat([newData,registration], axis = 1)
newData = pd.concat([newData,collaborator], axis = 1)
newData = pd.concat([newData,age], axis = 1)
newData = pd.concat([newData,companyTime], axis = 1)
newData = pd.concat([newData,one_hot_saida], axis = 1)
newData = pd.concat([newData,one_hot_cargos], axis = 1)
newData = pd.concat([newData,monthlySalary], axis = 1)
newData = pd.concat([newData,one_hot_genero], axis = 1)
newData = pd.concat([newData,one_hot_civil], axis = 1)
newData = pd.concat([newData,df_esco_atual], axis = 1)
newData = pd.concat([newData,one_hot_estado], axis = 1)
newData = pd.concat([newData,one_hot_cidade], axis = 1)
newData = pd.concat([newData,one_hot_area], axis = 1)

# ------------------------------------Tabela Reconhecimento--------------------------

# Cria uma tabela nova com os dados formatados
newData_reconhecimento = pd.DataFrame()
newData_reconhecimento = pd.concat([df_reconhecimento,one_hot_codinome], axis = 1)
newData_reconhecimento = pd.concat([df_reconhecimento,one_hot_situacao], axis = 1)
newData_reconhecimento = pd.concat([df_reconhecimento,one_hot_novocargo], axis = 1)
newData_reconhecimento = pd.concat([df_reconhecimento,one_hot_motivo], axis = 1)
newData_reconhecimento = pd.concat([df_reconhecimento,one_hot_funcao], axis = 1)


# ------------------------------------Tabela Reconhecimento--------------------------

# Cria uma tabela nova com os dados formatados
newData_ambiente = pd.DataFrame()
newData_ambiente = pd.concat([df_ambiente,one_hot_div], axis = 1)
newData_ambiente = pd.concat([df_ambiente,one_hot_pilar], axis = 1)
newData_ambiente = pd.concat([df_ambiente,punctuation], axis = 1)
newData_ambiente = pd.concat([df_ambiente,one_hot_factor], axis = 1)
newData_ambiente = pd.concat([df_ambiente,subpunctuation], axis = 1)
newData_ambiente = pd.concat([df_ambiente,one_hot_question], axis = 1)
newData_ambiente = pd.concat([df_ambiente,searchSkip], axis = 1)
newData_ambiente = pd.concat([df_ambiente,veryUnsatisfied], axis = 1)
newData_ambiente = pd.concat([df_ambiente,unsatisfied], axis = 1)
newData_ambiente = pd.concat([df_ambiente,neutral], axis = 1)
newData_ambiente = pd.concat([df_ambiente,satisfied], axis = 1)
newData_ambiente = pd.concat([df_ambiente,verySatisfied], axis = 1)
newData_ambiente = pd.concat([df_ambiente,one_hot_reliabilityRate], axis = 1)

# 7.Download das novas tabelas

Para verificação e facilidade de análise vamos fazer o download das tabelas.

In [80]:
newData.to_excel('formatData_Everymind.xlsx')
newData_reconhecimento.to_excel('formatData_Reconhecimento.xlsx')
newData_ambiente.to_excel('formatData_Ambiente.xlsx')